In [1]:
import csv
import numpy as np 

In [2]:
###  Load data

def read_csv(file):
    with open(file, newline='') as f:
        reader = csv.reader(f)
        line_count = 0
        rows = []
        for row in reader:
            if line_count == 0:
                titles = row
            else:
                rows.append(row)
            line_count += 1
    rows_int = np.array([[int(r) for r in row] for row in rows])
    return titles, rows_int
        

titles, rows_train = read_csv('data/studentsdigits-train.csv')
assert titles[-1] == 'Digit' and len(titles) == 9, 'Not train set'
X_train = rows_train[:,0:len(titles)-1]
Y_train = rows_train[:,-1]
print(X_train.shape)
print(Y_train.shape)

titles, rows_test = read_csv('data/studentsdigits-test.csv')
assert len(titles) == 8, 'Not test set'
X_test = rows_test
print(X_test.shape)

(3747, 8)
(3747,)
(3747, 8)


In [ ]:
# Build the MLP model


In [ ]:
### You code

### The predictions of test set: Y_test
Y_test = []

In [ ]:
### Save prediction results
assert len(Y_test) == len(X_test), 'sizes dont match'
with open('upload_predictions.txt', 'w') as fp:
    fp.write('\n'.join(str(y) for y in Y_test))
print('SAVED')